In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function

import lightgbm as lgb
import numpy as np
import pandas as pd
import csv
import os

from astropy.io import fits

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, LabelEncoder

import matplotlib.pyplot as plt

from timeit import default_timer as timer

import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, GlobalAveragePooling2D, Flatten, Dropout
from keras.optimizers import Adam
#from keras.callbacks import ModelCheckpoint, LearningRateScheduler
#from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10

from PIL import Image

import sdss_gz_data as sgd

/Users/will/Development/Astronomy/proposal/ENV/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
Using TensorFlow backend.


In [3]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalisation=True,
                 conv_first=True
                ):

    def apply_normalisation(x):
        if batch_normalisation:
            x = BatchNormalization()(x)
        
        return x
    
    def apply_activation(x):
        if activation is not None:
            x = Activation(activation)(x)
            
        return x
    
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
#                   kernel_regularizer=l2(1e-4),
                 )
    
    x = inputs
    if conv_first:
        x = apply_activation(apply_normalisation(conv(x)))
    else:
        x = conv(apply_activation(apply_normalisation(x)))
    
    return x

def get_config(stage, res_block, num_filters_in):
    num_filters_out = num_filters_in * 2
    activation = 'relu'
    batch_normalisation = True
    strides = 1
    
    if stage == 0:
        num_filters_out = num_filters_in * 4
        if res_block == 0:
            activation = None
            batch_normalisation = False
    else:
        if res_block == 0:
            strides = 2
            
    return num_filters_out, activation, batch_normalisation, strides

def main_block(inputs, num_filters_in, num_filters_out, strides, activation, batch_normalisation):
    y = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     kernel_size=1,
                     strides=strides,
                     activation=activation,
                     batch_normalisation=batch_normalisation,
                     conv_first=False)
    y = resnet_layer(inputs=y,
                     num_filters=num_filters_in,
                     conv_first=False)
    y = resnet_layer(inputs=y,
                     num_filters=num_filters_out,
                     kernel_size=1,
                     conv_first=False)
    
    return y

def residual_block(inputs, num_filters, strides):
    return resnet_layer(inputs=inputs,
                        num_filters=num_filters,
                        kernel_size=1,
                        strides=strides,
                        activation=None,
                        batch_normalisation=None
                       )

def resnetV2(inputs, depth, data_format='channels_last'):
    if (depth - 2) % 9 != 0:
        raise ValueError('Invalid depth, must be 9n+2')
    
    num_filters = 16
    num_res_blocks = int((depth - 2)/9)
    
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters,
                     conv_first=True,
                     
                    )
    
    for stage in range(3):
        for res_block in range(num_res_blocks):
            num_filters_out, activation, batch_normalisation, strides = get_config(stage, res_block, num_filters)
            
            y = main_block(inputs=x,
                           num_filters_in=num_filters,
                           num_filters_out=num_filters_out,
                           strides=strides,
                           activation=activation,
                           batch_normalisation=batch_normalisation
                          )
            
            if res_block == 0:
                x = residual_block(x, num_filters_out, strides)
                
            x = keras.layers.add([x, y])
        
        num_filters = num_filters_out
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return GlobalAveragePooling2D(data_format=data_format)(x)            

In [4]:
IMAGE_SIZE = 42
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
num_classes = 2

n_stages = 2
depth = 9 * n_stages + 2

In [5]:
from sdss_gz_data import SPIRIAL_GALAXY_TYPE
from sdss_gz_data import ELLIPTICAL_GALAXY_TYPE
from sdss_gz_data import UNKNOWN_GALAXY_TYPE
from sdss_gz_data import CONFIDENCE_LEVEL

object_cols = [
    'objid',
    'run',
    'rerun',
    'field',
    'camcol'
]

features = [
    'deVAB_i',
    'expAB_g',
    'expAB_i',
    'expAB_z',
    'expRad_g',
    'expRad_u',
    'expRad_z',
    'fiberMag_g',
    'fiberMag_u',
    'fiberMag_z',
    'model_g_u_colour_index',
    'model_i_r_colour_index',
    'model_r_g_colour_index',
    'model_z_i_colour_index',
    'petroRad_r',
    'petro_R90_R50_ratio_g',
    'petro_R90_R50_ratio_i',
    'petro_r_g_colour_index',
    'psfMag_r'    
]

all_cols = object_cols + features
print(all_cols)

['objid', 'run', 'rerun', 'field', 'camcol', 'deVAB_i', 'expAB_g', 'expAB_i', 'expAB_z', 'expRad_g', 'expRad_u', 'expRad_z', 'fiberMag_g', 'fiberMag_u', 'fiberMag_z', 'model_g_u_colour_index', 'model_i_r_colour_index', 'model_r_g_colour_index', 'model_z_i_colour_index', 'petroRad_r', 'petro_R90_R50_ratio_g', 'petro_R90_R50_ratio_i', 'petro_r_g_colour_index', 'psfMag_r']


In [6]:
orig_data = sgd.load_data('data/astromonical_data.csv.gz')

In [7]:
prepared_data = sgd.prepare_data(orig_data)

Number of high z galaxies = 231
Filtered out 3732 invalid records
% elliptical:      0.13576908942272356
% spiral:          0.2237143092857732
% unknown:         0.6405166012915032
% spiral of known: 0.6223216707350149


In [17]:
len(sgd.generate_features(use_averages=True))

113

In [8]:
transformed_data = sgd.transform_data(prepared_data)

In [9]:
X = transformed_data[all_cols]
y = transformed_data[['galaxy_type','z']]

In [10]:
len(transformed_data)

533332

In [11]:
known_galaxy_type_idx = transformed_data.galaxy_type != UNKNOWN_GALAXY_TYPE

X = X[known_galaxy_type_idx]
y = y[known_galaxy_type_idx]

In [12]:
len(X)

191724

In [15]:
len(y[y.galaxy_type == 0]), len(y[y.galaxy_type == 1])

(119314, 72410)

In [ ]:
X_train, X_test, y_train, y_test = sgd.split_train(X, y, test_size=0.2, random_state=42)
x_scaler = StandardScaler()
x_scaler.fit(X_train[features])

In [ ]:
selector = np.any([
    X_train.index == 36370,
    X_train.index == 25996,
    X_train.index == 9620,
    X_train.index == 519588,
    X_train.index == 481146,
    X_train.index == 60628,
    X_train.index == 480839,
    X_train.index == 480087
], axis=0)

X_train_small = X_train[selector]
y_train_small = y_train[selector]

In [ ]:
X_train_small

In [ ]:
from sdss_gz_data import redshift_err
from sdss_gz_data import z_err
from sdss_gz_data import z_err_stats

In [ ]:
from google.cloud import storage

gcs_client = storage.Client()
bucket = gcs_client.get_bucket('wgauvin-astroml-ast80014')

In [ ]:
# download fits image
def download_img(record, bucket):
    run = record.run
    camcol = record.camcol
    field = record.field
    objid = record.objid
    
    blob_dir = f'fits/{run}/{camcol}/{field}'
    filename = f'obj-{objid}.fits.bz2'
    blob_path = f'{blob_dir}/{filename}'
    
    print(f'Downloading {blob_path}')
    
    blob = bucket.get_blob(blob_path)
    blob.download_to_filename(filename)

    fits_file = fits.open(filename)
    
    return fits_file[0].data

def augment_image(data):
    return data

def crop_image(data, image_size):
    top_left = (72 - image_size)/2
    bottom_right = top_left + image_size
    
    output_data = np.zeros((3, image_size, image_size))
    
    for idx in range(3):
        img = Image.fromarray(data[idx])
        img = img.crop((top_left, top_left, bottom_right, bottom_right))
        output_data[idx] = np.array(img)
    
    return np.moveaxis(output_data, 0, -1)

def get_image(record, bucket):
    data = download_img(record, bucket)
    data = augment_image(data)
    return crop_image(data, IMAGE_SIZE)

In [ ]:
from astropy.coordinates import SkyCoord, ICRS
import astropy.units as u
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
from astropy.units import Quantity

In [ ]:
def normalise_images(images):
    min_ = np.min(images)
    max_ = np.max(images)

    images = (images - min_)/(max_ - min_)
    print(np.min(images), np.max(images))
    
    return images

def generate_training_data(X_train, y_train, use_cnn=True, use_features=True):
    def split(X, y):
        X_t = {
            'input_1': X['input_1'][0:6],
            'input_2': X['input_2'][0:6]
        }
        X_v = {
            'input_1': X['input_1'][6:8],
            'input_2': X['input_2'][6:8]
        }
        y_t = {
            'output_1': y['output_1'][0:6],
            'output_2': y['output_2'][0:6],
        }
        y_v = {
            'output_1': y['output_1'][6:8],
            'output_2': y['output_2'][6:8],
        }
        
        return X_t, y_t, X_v, y_v
        
    X = {  }
    y = { 'output_1': y_train['galaxy_type'], 'output_2': y_train['z'] }
    
    if use_cnn:
        images = np.ndarray((len(X_train), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=float)
    
        for idx, record in enumerate(X_train.itertuples()):
            images[idx] = get_image(record, bucket)

        X['input_1'] = normalise_images(images)
        
    if use_features:
        X['input_2'] = x_scaler.transform(X_train[features])

    return split(X, y)

In [ ]:
def create_nn(input_1_shape, input_2_shape, dense_units=512, lr=0.00003, dropout=True, use_cnn=True, use_features=True):
    if use_cnn:
        input_1 = Input(shape=input_1_shape)
        cnn = resnetV2(input_1, depth)

    if use_features:
        input_2 = Input(shape=(input_2_shape,), name='input_2')

    if use_cnn:
        if use_features:
            x = keras.layers.concatenate([cnn, input_2])
        else:
            x = cnn
    else:
        if use_features:
            x = input_2
        else:
            raise Exception('Need at least one input')
    
    # Make sure we normalise after concatination!!!
    if (use_features and use_cnn):
        x = BatchNormalization()(x)

    x = Dense(dense_units,
              kernel_initializer='random_normal',
              name='hidden_layer_1',
              use_bias=True,
              activation='relu'
             )(x)
    x = BatchNormalization()(x)
    
    if (dropout):
        x = Dropout(0.1)(x)

    x = Dense(dense_units,
              kernel_initializer='random_normal',
              name='hidden_layer_2',
              use_bias=True,
              activation='relu'
             )(x)
    x = BatchNormalization()(x)

    if (dropout):
        x = Dropout(0.3)(x)

    output_1 = Dense(1, # classification
                     kernel_initializer='random_normal',
                     name='output_1',
                     use_bias=True,
                     activation='sigmoid', 
                    )(x)
    output_2 = Dense(1,
                     kernel_initializer='random_normal',
                     name='output_2',
                     use_bias=True,
                     activation='linear'
                    )(x)

    optimizer = Adam(lr=lr)
    
    outputs = [
        output_1,
        output_2
    ]

    if use_cnn:
        if use_features:
            inputs = [
                input_1,
                input_2
            ]
        else:
            inputs = [ input_1 ]
    else:
        inputs = [ input_2 ]

    model = Model(inputs=inputs, outputs=outputs)
    loss_weights = { 
        'output_1': 1.0,
        'output_2': 5.0
    }
    loss = {
        'output_1': 'binary_crossentropy',
        'output_2': redshift_err
    }
    metrics = {
        'output_1': 'accuracy',
        'output_2': redshift_err
    }
    
    model.compile(loss=loss,
                  optimizer=optimizer,
                  loss_weights=loss_weights,
                  metrics=metrics
                 )
    return model

#     output_1 = Dense(num_classes, name='output_1', activation='sigmoid', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(x)
#     output_2 = Dense(1, name='output_2', activation='linear', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(x)
#     return Model(inputs=[input_1, input_2], outputs=[output_1, output_2])
    
# K.clear_session()
# model = create_nn(input_shape, len(features), dense_units=256, lr=0.0001, dropout=False, use_cnn=False)
# model.summary()
# init_weights = model.get_weights()

In [ ]:
#K.clear_session()
#model.set_weights(init_weights)
#model = create_nn(input_shape, len(features), dense_units=256, lr=0.001, dropout=False, use_cnn=False)
# model.set_weights(init_weights)
# model.summary()
# model.fit(X_train_, y_train_, epochs=1000, batch_size=1)
# X_train_, y_train_ = generate_training_data(X_train_small[0:1], y_train_small[0:1], use_cnn=False, use_features=False)
#model.fit(X_train['input_2'], y_train['output_2'], epochs=10000, batch_size=1)

In [ ]:
def train(X, y, dropout=True, use_cnn=True, use_features=True, epochs=1000, batch_size=1, lr=0.1):
    K.clear_session()
    model = create_nn(input_shape,
                      len(features),
                      dense_units=256,
                      lr=lr,
                      dropout=dropout,
                      use_cnn=use_cnn,
                      use_features=use_features
                     )
    model.summary()
    init_weights = model.get_weights()
    
    X_t, y_t, X_v, y_v = generate_training_data(X,
                                                y,
                                                use_cnn=use_cnn,
                                                use_features=use_features
                                               )
    model.fit(X_t,
              y_t,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_v, y_v)
             )
    
    return model, init_weights, X_t, y_t, X_v, y_v

In [ ]:
model, init_weights, X_train_, y_train_, X_val_, y_val_ = train(
    X_train_small,
    y_train_small,
    use_cnn=True,
    use_features=True,
    epochs=100,
    lr=0.00003,
    batch_size=2
)

In [ ]:
result = model.predict(X_t)
result

In [ ]:
np.round(result[0]).ravel().astype(int)

In [ ]:
y_t['output_1'].values

In [ ]:
result[1].ravel()

In [ ]:
y_t['output_2'].values

In [ ]:
X_train_['input_1'].shape, X_train_['input_2'].shape, y_train_['output_1'].shape, y_train_['output_2'].shape

In [ ]:
X_t = {
    'input_1': X_train_['input_1'][0:6],
    'input_2': X_train_['input_2'][0:6]
}
X_v = {
    'input_1': X_train_['input_1'][6:8],
    'input_2': X_train_['input_2'][6:8]
}
y_t = {
    'output_1': y_train_['output_1'][0:6],
    'output_2': y_train_['output_2'][0:6],
}
y_v = {
    'output_1': y_train_['output_1'][6:8],
    'output_2': y_train_['output_2'][6:8],
}

X_t['input_1'].shape, X_t['input_2'].shape, y_t['output_1'].shape, y_t['output_2'].shape, X_v['input_1'].shape, X_v['input_2'].shape, y_v['output_1'].shape, y_v['output_2'].shape

In [ ]:
model.evaluate(X_t, y_t)

In [ ]:
y_train_

In [ ]:
np.min(X_train_['input_1']), np.max(X_train_['input_1'])

In [ ]:
result[1]

In [ ]:
(result[1][0] - y_train_['output_2'])/(1 + y_train_['output_2'])

In [ ]:
z_err_val = np.average(np.abs((y_train_['output_2'] - result[1][0])/(1 + y_train_['output_2'])))
z_err_val

In [ ]:
print(y_train_['output_2'].values, result[1])
z_err_stats(y_train_['output_2'].values, result[1])

In [ ]:
z_err(result[1], y_train_['output_2'])

In [ ]:
list_of_indexes = X_train.index.values

indexes = np.arange(len(list_of_indexes))

np.random.shuffle(indexes)
print(list_of_indexes, indexes)
list_of_indexes[indexes[0]]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

galaxy_image_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=360,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1
)

# need to crop data
def crop_image(image, image_size):
    

In [ ]:
print(type(X_train[X_train.index == 480839].to_records()))


In [ ]:
print(type(X_train.loc[480839]))
print(X_train.loc[480839]['objid'])

In [ ]:
X_train_small

In [ ]:
#X_train[X_train.index == 480839]
X_train_small.loc[480839]

In [ ]:
y.loc[480839]['galaxy_type']

In [ ]:
print(type(X_train_small.loc[480839]))
X_train_small.loc[480839].objid

In [ ]:
for idx, record in enumerate(X_train_small.itertuples()):
    print(record.objid)

In [ ]:
start = 0
end = 6

idx_range = range(6)
print(idx_range)


X_train_small.iloc[idx_range]

In [ ]:
len(idx_range)

In [ ]:
del range
for idx in range(6):
    print(idx)